In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

import requests
import lxml
from lxml.html import fromstring

from fake_useragent import UserAgent

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# headers
ua = UserAgent()
headers = {'User-Agent' : ua.random}

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument(f"user-agent={ua}")

# 파라미터

In [ ]:
KEYWORDS = ['헬멧','바퀴']

# URL 크롤링
 - time.sleep(2)로 했음에도 401 ERROR 발생,, TIME을 충분히 줄 필요성이 있음

In [ ]:
df = pd.DataFrame(columns=['키워드명','url'])
t1 = time.time()

for j, KEYWORD in enumerate(KEYWORDS):

    # 링크 제시
    base_url = f'http://emart.ssg.com/search.ssg?target=all&query={KEYWORD}&sort=cnt&count=100' # count=100 --> 100개만 수집!
    time.sleep(3)

    # 정보수집 
    res = requests.get(base_url, headers=headers)
    res.raise_for_status()
    res.encoding = "utf-8-sig"
    parser = fromstring(res.text)

    elements = parser.xpath('/html/body/div[4]/div[4]/div[2]/div[1]/div[5]/div[5]/div[1]/ul/li/div[2]/div[2]/div/a')
    urls = ['http://emart.ssg.com'+element.get('href') for element in elements]
    keywords = [KEYWORD] * len(urls)

    # 데이터프레임에 추가
    tmp = pd.DataFrame({'키워드명':keywords,
                        'url':urls})
    df = df.append(tmp, ignore_index=True)

    # log
    print(f'{j+1}번째 키워드 {KEYWORD} 완료!! == 경과시간: {round(time.time() - t1)}초 == 수집개수: {len(urls)}개')

# 세부 값들 크롤링

In [ ]:
# 키워드별로 url 을 리스트로 정리
df2 = df[['키워드명']].drop_duplicates().reset_index(drop=True)
df3 = pd.DataFrame()

for index_ in range(df2.shape[0]):

    keyword_tmp = df2.loc[index_, '키워드명']
    urls_tmp = df.loc[(df['키워드명'] == keyword_tmp),'url'].tolist()

    tmp_df = pd.DataFrame({'키워드명':[keyword_tmp],
                          'url':[urls_tmp]})
    df3 = df3.append(tmp_df, ignore_index=True)
df3

In [ ]:
# 값들 초기화
df_final = pd.DataFrame()
driver = webdriver.Chrome('chromedriver_91.exe', options=options)
t1 = time.time()
NO_REVIEW = False

for idx in range(df3.shape[0]):

    for i ,url in enumerate(df3.loc[idx,'url']):

        # url에 접근
        try:
            driver.get(url)   
        except:
            print('url에 접근할 수 없습니다. 다시 url을 open합니다. (10분휴식)')
            driver.close()
            time.sleep(600)
            driver = webdriver.Chrome('chromedriver_91.exe', options=options)
            driver.get(url)   


        time.sleep(2) ### SSG가 자꾸 URL접속에 민감한 것 같아서 추가,,
        print(f'{i+1}번째 url 시작!! == 경과시간: {round(time.time()-t1)}초')


        # 제품명 정보수집
        product = driver.find_element_by_class_name('cdtl_prd_info').find_element_by_class_name('cdtl_info_tit').text

        # 스크롤을 어느정도 내린 후 고객리뷰 클릭
        driver.execute_script(f"window.scrollTo(0, 2000)")     
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="_cdtl_dtlcont_wrap"]/div[1]/div/div[1]/ul/li[3]/a').click()
        time.sleep(0.5)

        # 고객리뷰 평점낮은순정렬
        location = driver.find_element_by_xpath('//*[@id="cmt_select_sort"]/div/a/span[2]')
        driver.execute_script(f"window.scrollTo(0, {location.location['y']-200})") 
        driver.find_element_by_xpath('//*[@id="cmt_select_sort"]/div/a/span[2]').click()
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="cmt_select_sort"]/div/div/ul/li[4]/a/span').click()

        # 리뷰관련 내부페이지 이동
        while True:
            
            time.sleep(2)

            ### 정보수집
            scores = [int(score.text) for score in driver.find_elements_by_xpath('//*[@id="cdtl_cmt_tbody"]/tr/td[2]/div/span/span/span/em')]
            dates = [date.text for date in driver.find_elements_by_xpath('//*[@id="cdtl_cmt_tbody"]/tr/td[5]/div')]
            writers = [writer.text for writer in driver.find_elements_by_xpath('//*[@id="cdtl_cmt_tbody"]/tr/td[4]/div')]
            contents = [content.text for content in driver.find_elements_by_xpath('//*[@id="cdtl_cmt_tbody"]/tr/td[3]/div/a/div[1]/span')]

            ### 데이터프레임화 및 추가
            tmp = pd.DataFrame({'제품명':[product]*len(writers),
                              '작성자':writers,
                              '작성일자':dates,
                              '평점':scores,
                              '본문':contents,
                              '검색키워드명':[df3.loc[idx,'키워드명']]*len(writers),
                              'url':[url]*len(writers)})

            df_final = df_final.append(tmp.drop_duplicates(), ignore_index=True)

            ### 페이지있는 숫자bar로 이동
            time.sleep(0.5)
            location = driver.find_elements_by_xpath('//*[@id="comment_navi_area"]/strong')

            ##### 리뷰가 없다면, while문 탈출 및 키워드 이동
            if len(location) == 0:
                print('리뷰가 없습니다.')
                NO_REVIEW = True
                break

            else:
                driver.execute_script(f"window.scrollTo(0, {location[0].location['y']-200})") 

            ### 페이지 이동
            page_now = int(driver.find_element_by_xpath('//*[@id="comment_navi_area"]/strong').text)
            page_dict = {element.text : element for element in driver.find_elements_by_xpath('//*[@id="comment_navi_area"]/a') if (element.text.isdigit()) | (element.text == '다음')}
            move_page = page_now + 1

            if '다음' in page_dict.keys():

                if move_page % 10 == 1:
                    page_dict['다음'].click()

                else:
                    page_dict[str(move_page)].click()

            else:
                if (len([int(key) for key in page_dict.keys()]) == 0):
                    print('모든 페이지에 도달했습니다. 해당 페이지 리뷰 수집을 종료합니다.')
                    break

                else:
                    if (page_now == max([int(key) for key in page_dict.keys()])+1):
                        print('모든 페이지에 도달했습니다. 해당 페이지 리뷰 수집을 종료합니다.')
                        break

                    else:
                        page_dict[str(move_page)].click()

            ### 수집종료조건: 평점 3이하만 수집한다.
            if max(scores) >= 4:
                print(f'{page_now} 페이지의 평점은 4점이상 입니다. 수집할 필요가 없다고 판단하여 종료합니다.')
                break


        if NO_REVIEW == True:
            print('리뷰가 더이상 없네요. 키워드를 변경합니다.') 
            print('키워드가 바뀌었습니다. 5초를 쉽니다.')
            time.sleep(5)
            break


    df_final = df_final[df_final['평점'] <= 3].drop_duplicates().reset_index(drop=True)
    print('크롤링 완료!!')
    
df_final = df_final[df_final['평점'] <= 3].drop_duplicates().reset_index(drop=True)
df_final

In [ ]:
df_final.to_excel('SSG_크롤링_샘플.xlsx', index=False)